## Modelado de Temas con LDA

### Instalación de Librerías

In [1]:
!pip install pandas gensim spacy nltk

### Implementación LDA

In [2]:
import json
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS as GENSIM_STOP
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel, Phrases
from gensim.models.phrases import Phraser
from tqdm import tqdm

# -------------------------------------------------
# 1. Descargas necesarias (solo la primera vez)
# -------------------------------------------------
nltk.download('stopwords', quiet=True)

# Definir stopwords en español y stemmer
nltk_stop = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')


# -------------------------------------------------
# 2. Función de preprocesado (tokenización + stemming)
# -------------------------------------------------
def preprocess(texto: str) -> list[str]:
    """
    1) Convierte a minúsculas y tokeniza quitando puntuación y acentos (deacc=True).
    2) Filtra tokens de longitud <= 3.
    3) Elimina stopwords de gensim y NLTK.
    4) Aplica stemming en español.
    Devuelve la lista de stems resultantes.
    """
    tokens = simple_preprocess(texto, deacc=True)
    tokens = [t for t in tokens if len(t) > 3 and t not in GENSIM_STOP and t not in nltk_stop]
    stems = [stemmer.stem(t) for t in tokens]
    return stems


# -------------------------------------------------
# 3. (Opcional) Generación de bigramas
# -------------------------------------------------
def aplicar_bigramas(
        documentos: list[list[str]],
        min_count: int = 5,
        threshold: float = 100.0
) -> tuple[list[list[str]], Phraser]:
    """
    1) Entrena un modelo de bigramas sobre los documentos tokenizados.
    2) Devuelve los documentos enriquecidos con bigramas (uni_bi) y el modelo Phraser.
    Parámetros:
      - min_count: número mínimo de apariciones conjuntas para considerar bigrama.
      - threshold: umbral de formación de bigramas (valor alto = más conservador).
    """
    bigram = Phrases(documentos, min_count=min_count, threshold=threshold, progress_per=10000)
    bigram_mod = Phraser(bigram)
    documentos_bi = [bigram_mod[doc] for doc in documentos]
    return documentos_bi, bigram_mod


# -------------------------------------------------
# 4. Construcción de diccionario y corpus (BoW)
# -------------------------------------------------
def construir_diccionario_corpus(
        textos_tokenizados: list[list[str]],
        no_below: int = 5,
        no_above: float = 0.5
) -> tuple[corpora.Dictionary, list]:
    """
    1) Crea un Dictionary a partir de la lista de documentos (lista de tokens).
    2) Filtra palabras muy raras (no_below) o demasiado frecuentes (no_above).
    3) Construye el corpus en formato BoW (lista de tuplas (id_token, frecuencia)).
    Devuelve el diccionario filtrado y el corpus BoW.
    """
    diccionario = corpora.Dictionary(textos_tokenizados)
    diccionario.filter_extremes(no_below=no_below, no_above=no_above)
    corpus = [diccionario.doc2bow(doc) for doc in textos_tokenizados]
    return diccionario, corpus


# -------------------------------------------------
# 5. Entrenamiento y evaluación del modelo LDA
# -------------------------------------------------
def entrenar_lda(
        corpus: list,
        diccionario: corpora.Dictionary,
        num_topics: int = 7,
        passes: int = 10,
        random_state: int = 42,
        alpha: str = 'auto'
) -> LdaModel:
    """
    Entrena un modelo LDA con los parámetros indicados y devuelve el objeto LdaModel.
    """
    lda_model = LdaModel(
        corpus=corpus,
        id2word=diccionario,
        num_topics=num_topics,
        random_state=random_state,
        passes=passes,
        alpha=alpha,
        per_word_topics=True
    )
    return lda_model


def evaluar_lda(
        lda_model: LdaModel,
        corpus: list,
        textos_tokenizados: list[list[str]],
        diccionario: corpora.Dictionary
) -> tuple[float, float]:
    """
    1) Calcula la perplejidad del modelo sobre el corpus.
    2) Calcula la coherencia (c_v) usando CoherenceModel.
    Devuelve (perplejidad, coherencia).
    """
    perplejidad = lda_model.log_perplexity(corpus)
    coh_model = CoherenceModel(
        model=lda_model,
        texts=textos_tokenizados,
        dictionary=diccionario,
        coherence='c_v'
    )
    coherencia = coh_model.get_coherence()
    return perplejidad, coherencia


# -------------------------------------------------
# 6. Asignación de tópico dominante a cada documento
# -------------------------------------------------
def asignar_topic_principal(lda_model: LdaModel, corpus: list) -> list[int]:
    """
    Para cada documento en formato BoW, obtiene la lista de tópicos con sus probabilidades
    y retorna el ID del tópico con mayor probabilidad (dominante).
    """
    topicos_principales = []
    for bow in corpus:
        distribucion = lda_model.get_document_topics(bow, minimum_probability=0.0)
        topico_dom = max(distribucion, key=lambda x: x[1])[0]
        topicos_principales.append(topico_dom)
    return topicos_principales


# -------------------------------------------------
# 7. Leer JSON y preparar DataFrame con 'description' y 'category'
# -------------------------------------------------
def cargar_descriptions_desde_json(ruta_json: str) -> pd.DataFrame:
    """
    Lee un archivo JSON con estructura de lista de objetos:
    [
      {
        "title": "...",
        "category": "...",
        "summit": "...",
        "description": "...",
        "date": "...",
        "autor": "...",
        "tags": "['Seguro Social', 'Estados Unidos']",
        "url": "..."
      },
      ...
    ]
    Extrae las columnas 'description' y 'category', elimina filas con description vacía.
    Devuelve un DataFrame con dichas columnas.
    """
    # Cargar JSON en un DataFrame
    df = pd.read_json(ruta_json, orient='records', encoding='utf-8')
    # Filtrar filas donde 'description' exista y no esté vacío
    df = df[df['description'].notna() & (df['description'].str.strip() != "")].reset_index(drop=True)
    return df[['description', 'category']]

### Identificación de 7 tópicos mediante LDA y evaluación del modelo

In [3]:
# -------------------------------------------------
# 8. Función principal: pipeline completo adaptado a JSON
# Parámetros ajustables
RUTA_JSON = "gestionspider4.json"        # Nombre del JSON a leer
NUM_TOPICS = 7                     # Número de tópicos para LDA
PASSES = 10                        # Número de pasadas (iteraciones) en el entrenamiento LDA
NO_BELOW = 5                       # Filtrar tokens que aparezcan en menos de NO_BELOW documentos
NO_ABOVE = 0.5                     # Filtrar tokens que aparezcan en más del 50% de documentos
BIGRAM = True                      # Si quieres incluir n-gramas (True o False)
BIGRAM_MIN_COUNT = 5               # Mínimo de apariciones para formar un bigrama
BIGRAM_THRESHOLD = 100.0           # Umbral de formación de bigramas

# 1) Leer el JSON y extraer 'description' y 'category'
df = cargar_descriptions_desde_json(RUTA_JSON)
print(f"Cargados {len(df)} registros desde '{RUTA_JSON}'.\n")

# 2) Preprocesar los textos en 'description'
textos_raw = df['description'].astype(str).tolist()
documentos_tokenizados = []
print("Preprocesando descripciones (tokenización + stemming)...")
for doc in tqdm(textos_raw, desc="Progreso preprocesado"):
    documentos_tokenizados.append(preprocess(doc))

# 3) (Opcional) Generar e incluir bigramas
if BIGRAM:
    print("\nDetectando bigramas en el corpus...")
    documentos_bi, modelo_bigram = aplicar_bigramas(
        documentos_tokenizados,
        min_count=BIGRAM_MIN_COUNT,
        threshold=BIGRAM_THRESHOLD
    )
    textos_finales = documentos_bi
else:
    textos_finales = documentos_tokenizados

# 4) Construir diccionario y corpus BoW
print("\nConstruyendo diccionario y corpus BoW...")
diccionario, corpus = construir_diccionario_corpus(
    textos_tokenizados=textos_finales,
    no_below=NO_BELOW,
    no_above=NO_ABOVE
)
print(f"Diccionario creado: {len(diccionario)} tokens únicos.\n")

# 5) Entrenar modelo LDA
print(f"Entrenando LDA con {NUM_TOPICS} tópicos (passes={PASSES})...")
lda_model = entrenar_lda(
    corpus=corpus,
    diccionario=diccionario,
    num_topics=NUM_TOPICS,
    passes=PASSES
)
print("Modelo LDA entrenado.\n")

# 6) Evaluar modelo (perplejidad y coherencia)
print("Evaluando modelo LDA...")
perplejidad, coherencia = evaluar_lda(
    lda_model=lda_model,
    corpus=corpus,
    textos_tokenizados=textos_finales,
    diccionario=diccionario
)
print(f"► Perplejidad: {perplejidad:.4f}")
print(f"► Coherencia (c_v): {coherencia:.4f}\n")

# 7) Mostrar top-10 palabras de cada tópico
print("Términos más representativos por tópico:")
for tid in range(NUM_TOPICS):
    términos = lda_model.show_topic(tid, topn=10)
    lista_palabras = ", ".join([palabra for palabra, _ in términos])
    print(f"  Tópico {tid}: {lista_palabras}")
print()

# 8) Asignar tópico dominante a cada documento
print("Asignando tópico dominante a cada documento...")
df['bow'] = corpus
df['topic_principal'] = asignar_topic_principal(lda_model, corpus)

Cargados 3698 registros desde 'gestionspider4.json'.

Preprocesando descripciones (tokenización + stemming)...


Progreso preprocesado: 100%|██████████| 3698/3698 [00:07<00:00, 475.32it/s]



Detectando bigramas en el corpus...

Construyendo diccionario y corpus BoW...
Diccionario creado: 8588 tokens únicos.

Entrenando LDA con 7 tópicos (passes=10)...
Modelo LDA entrenado.

Evaluando modelo LDA...
► Perplejidad: -7.5791
► Coherencia (c_v): 0.4100

Términos más representativos por tópico:
  Tópico 0: empres, trabaj, peru, inversion, sector, millon, public, mayor, oper, indic
  Tópico 1: president, unid, pais, trump, gobiern, estadounidens, dij, part, segur, segun
  Tópico 2: cas, public, pued, inform, deb, present, congres, part, proces, nacional
  Tópico 3: pued, empres, sol, trabaj, anos, hac, merc, peru, person, mejor
  Tópico 4: product, merc, preci, export, arancel, unid, chin, pais, podri, import
  Tópico 5: part, hor, nuev, pued, mexic, unid, pais, ser, canal, encuentr
  Tópico 6: proyect, millon, oper, nuev, inversion, peru, lim, empres, marc, desarroll

Asignando tópico dominante a cada documento...


### Identificación del tópico a 5 noticias

In [4]:
# 9) Mostrar las primeras filas con su categoría original y tópico asignado
print("\nEjemplo de asignación de tópicos:")
print(df[['description', 'category', 'topic_principal']].head(5))


Ejemplo de asignación de tópicos:
                                         description   category  \
0  Hace 26 años se iniciaron las concesiones de p...   Economía   
1  El billete verde vuelve a cerrar un mes a la b...  Tu Dinero   
2  La Victoria es uno de los distritos más resalt...   Empresas   
3  Las alarmas permanecen encendidas en el sector...   Economía   
4  El ahorro de los peruanos se sigue expandiendo...  Tu Dinero   

   topic_principal  
0                6  
1                4  
2                6  
3                0  
4                0  


In [7]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS as GENSIM_STOP
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel, Phrases, TfidfModel
from gensim.models.phrases import Phraser
from tqdm import tqdm

# -------------------------------------------------
# 1. Descargas NLTK (solo la primera vez)
# -------------------------------------------------
nltk.download('stopwords', quiet=True)

# -------------------------------------------------
# 2. Definir stopwords en español y stemmer
# -------------------------------------------------
nltk_stop = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')


# -------------------------------------------------
# 3. Función de preprocesado (tokenización + stemming)
# -------------------------------------------------
def preprocess(texto: str) -> list[str]:
    """
    1) Convierte a minúsculas y tokeniza quitando puntuación y acentos (deacc=True).
    2) Filtra tokens de longitud <= 3.
    3) Elimina stopwords de gensim y NLTK.
    4) Aplica stemming en español.
    Devuelve la lista de stems resultantes.
    """
    tokens = simple_preprocess(texto, deacc=True)
    tokens = [t for t in tokens if len(t) > 3 and t not in GENSIM_STOP and t not in nltk_stop]
    stems = [stemmer.stem(t) for t in tokens]
    return stems


# -------------------------------------------------
# 4. Generación de bigramas (opcional)
# -------------------------------------------------
def aplicar_bigramas(documentos: list[list[str]],
                     min_count: int = 5,
                     threshold: float = 100.0) -> tuple[list[list[str]], Phraser]:
    """
    1) Entrena un modelo de bigramas sobre los documentos tokenizados.
    2) Devuelve los documentos enriquecidos con bigramas (uni_bi) y el modelo Phraser.
    """
    bigram = Phrases(documentos, min_count=min_count, threshold=threshold, progress_per=10000)
    bigram_mod = Phraser(bigram)
    documentos_bi = [bigram_mod[doc] for doc in documentos]
    return documentos_bi, bigram_mod


# -------------------------------------------------
# 5. Construcción de diccionario y corpus BoW
# -------------------------------------------------
def construir_diccionario_corpus(textos_tokenizados: list[list[str]],
                                 no_below: int = 5,
                                 no_above: float = 0.5) -> tuple[corpora.Dictionary, list]:
    """
    1) Crea un Dictionary a partir de la lista de documentos (lista de tokens).
    2) Filtra palabras muy raras (no_below) o demasiado frecuentes (no_above).
    3) Construye el corpus en formato BoW (lista de tuplas (id_token, frecuencia)).
    Devuelve el diccionario filtrado y el corpus BoW.
    """
    diccionario = corpora.Dictionary(textos_tokenizados)
    diccionario.filter_extremes(no_below=no_below, no_above=no_above)
    corpus_bow = [diccionario.doc2bow(doc) for doc in textos_tokenizados]
    return diccionario, corpus_bow


# -------------------------------------------------
# 6. Entrenamiento y evaluación del modelo LDA
# -------------------------------------------------
def entrenar_lda(corpus: list, diccionario: corpora.Dictionary,
                 num_topics: int = 7,
                 passes: int = 10,
                 random_state: int = 42,
                 alpha: str = 'auto') -> LdaModel:
    """
    Entrena un modelo LDA con los parámetros indicados y devuelve el objeto LdaModel.
    """
    lda_model = LdaModel(
        corpus=corpus,
        id2word=diccionario,
        num_topics=num_topics,
        random_state=random_state,
        passes=passes,
        alpha=alpha,
        per_word_topics=True
    )
    return lda_model


def evaluar_lda(lda_model: LdaModel,
                corpus: list,
                textos_tokenizados: list[list[str]],
                diccionario: corpora.Dictionary) -> tuple[float, float]:
    """
    1) Calcula la perplejidad del modelo sobre el corpus.
    2) Calcula la coherencia (c_v) usando CoherenceModel.
    Devuelve (perplejidad, coherencia).
    """
    perplejidad = lda_model.log_perplexity(corpus)
    coh_model = CoherenceModel(model=lda_model,
                               texts=textos_tokenizados,
                               dictionary=diccionario,
                               coherence='c_v')
    coherencia = coh_model.get_coherence()
    return perplejidad, coherencia


# -------------------------------------------------
# 7. Asignación de tópico dominante a cada documento
# -------------------------------------------------
def asignar_topic_principal(lda_model: LdaModel, corpus: list) -> list[int]:
    """
    Para cada documento en formato BoW o TF-IDF, obtiene la lista de tópicos
    con sus probabilidades y retorna el ID del tópico con mayor probabilidad.
    """
    topicos_principales = []
    for doc_vector in corpus:
        distribucion = lda_model.get_document_topics(doc_vector, minimum_probability=0.0)
        topico_dom = max(distribucion, key=lambda x: x[1])[0]
        topicos_principales.append(topico_dom)
    return topicos_principales


# -------------------------------------------------
# 8. Función principal: pipeline completo
# -------------------------------------------------
def main():
    # Parámetros ajustables
    RUTA_CSV = "gestionspider4.json"      # Nombre del CSV a leer
    NUM_TOPICS = 7                    # Número de tópicos para LDA
    PASSES = 10                       # Número de pasadas en LDA
    NO_BELOW = 5                      # Mínimo de documentos para que un token sobreviva
    NO_ABOVE = 0.5                    # Máximo porcentaje de documentos para que un token sobreviva
    BIGRAM = True                     # Si generar bigramas (True/False)
    BIGRAM_MIN_COUNT = 5              # Umbral mínimo de apariciones conjuntas para bigramas
    BIGRAM_THRESHOLD = 100.0          # Umbral de formación de bigramas

    # 1) Leer el CSV (solo columnas 'summit' y 'category')
    df = pd.read_csv(RUTA_CSV, delimiter='|', encoding='utf-8', usecols=['summit', 'category'])
    print(f"Cargados {len(df)} registros desde '{RUTA_CSV}'.\n")

    # 2) Preprocesar los textos (tokenización + stemming) con tqdm
    textos_raw = df['summit'].astype(str).tolist()
    documentos_tokenizados = []
    print("1) Preprocesando textos (tokenización + stemming)...")
    for doc in tqdm(textos_raw, desc="Preprocesado"):
        documentos_tokenizados.append(preprocess(doc))

    # 3) (Opcional) Generar e incluir bigramas
    if BIGRAM:
        print("\n2) Detectando bigramas en el corpus...")
        documentos_bi, modelo_bigram = aplicar_bigramas(
            documentos_tokenizados,
            min_count=BIGRAM_MIN_COUNT,
            threshold=BIGRAM_THRESHOLD
        )
        textos_finales = documentos_bi
    else:
        textos_finales = documentos_tokenizados

    # 4) Construir diccionario y corpus BoW
    print("\n3) Construyendo diccionario y corpus BoW...")
    diccionario, corpus_bow = construir_diccionario_corpus(
        textos_tokenizados=textos_finales,
        no_below=NO_BELOW,
        no_above=NO_ABOVE
    )
    print(f"   - Diccionario creado con {len(diccionario)} tokens únicos.\n")

    # -------------------------------------------------
    # 5) CÁLCULO DE TF-IDF
    # -------------------------------------------------
    print("4) Calculando modelo TF-IDF sobre el corpus BoW...")
    tfidf_model = TfidfModel(corpus_bow, dictionary=diccionario)
    corpus_tfidf = tfidf_model[corpus_bow]

    # 5.1) EXTRAER PALABRAS MÁS RELEVANTES (suma de TF-IDF por token)
    print("   → Extrayendo las palabras más relevantes (sumando TF-IDF por token)...")
    # Creamos un vector global que acumule TF-IDF de cada token sobre todos los docuementos
    tfidf_global = {}
    for doc in corpus_tfidf:
        for token_id, tfidf_val in doc:
            if token_id not in tfidf_global:
                tfidf_global[token_id] = tfidf_val
            else:
                tfidf_global[token_id] += tfidf_val

    # Ordenamos todos los tokens por peso TF-IDF acumulado descendente
    tokens_ordenados = sorted(tfidf_global.items(), key=lambda x: x[1], reverse=True)
    TOP_TFIDF = 20
    print(f"\n   Top {TOP_TFIDF} tokens más relevantes (TF-IDF acumulado):")
    for idx, (token_id, acumulado) in enumerate(tokens_ordenados[:TOP_TFIDF], start=1):
        palabra = diccionario[token_id]
        print(f"     {idx:>2}. {palabra:<20}  —  TF-IDF acumulado: {acumulado:.4f}")
    print()

    # -------------------------------------------------
    # 6) ENTRENAR LDA sobre CORPUS TF-IDF
    # -------------------------------------------------
    print("5) Entrenando LDA (7 tópicos) sobre el corpus TF-IDF...")
    lda_model = entrenar_lda(
        corpus=list(corpus_tfidf),   # IMPORTANTE: usamos corpus_tfidf en lugar de corpus_bow
        diccionario=diccionario,
        num_topics=NUM_TOPICS,
        passes=PASSES
    )
    print("   ► LDA entrenado.\n")

    # 6.1) Evaluar modelo (perplejidad y coherencia)
    print("6) Evaluando modelo LDA...")
    perplejidad, coherencia = evaluar_lda(
        lda_model=lda_model,
        corpus=list(corpus_tfidf),
        textos_tokenizados=textos_finales,
        diccionario=diccionario
    )
    print(f"   ► Perplejidad: {perplejidad:.4f}")
    print(f"   ► Coherencia (c_v): {coherencia:.4f}\n")

    # 6.2) Mostrar top-10 palabras de cada tópico
    print("7) Términos más representativos por tópico (Top 10):")
    for tid in range(NUM_TOPICS):
        términos = lda_model.show_topic(tid, topn=10)
        términos_str = ", ".join([pal for pal, _ in términos])
        print(f"   Tópico {tid}: {términos_str}")
    print()

    # -------------------------------------------------
    # 7) ASIGNAR TÓPICO DOMINANTE A CADA DOCUMENTO
    # -------------------------------------------------
    print("8) Asignando tópico dominante a cada noticia...")
    df['bow'] = corpus_bow
    # NOTA: La función `asignar_topic_principal` funciona con corpus en BoW o en TF-IDF
    df['topic_principal'] = asignar_topic_principal(lda_model, list(corpus_tfidf))
    print("   → Asignación completada.\n")

    # -------------------------------------------------
    # 8) MOSTRAR RESULTADOS PARA 3 NOTICIAS EJEMPLO
    # -------------------------------------------------
    print("9) Mostrando resultados de LDA para 3 noticias de ejemplo:\n")
    # Seleccionamos tres índices arbitrarios (por ejemplo 0, 10 y 20) o aleatorios
    ejemplos_idx = [0, 10, 20]
    for idx in ejemplos_idx:
        texto_original = df.loc[idx, 'summit']
        categoria = df.loc[idx, 'category']
        bow         = df.loc[idx, 'bow']
        tfidf_vec   = corpus_tfidf[idx]
        topico_dom  = df.loc[idx, 'topic_principal']

        # Obtenemos la distribución completa de tópicos para ese documento
        distribucion = lda_model.get_document_topics(tfidf_vec, minimum_probability=0.0)

        print(f"→ Noticia #{idx} (categoría = '{categoria}'):")
        print(f"   * Texto original (resumen): {texto_original[:100]}...")  # mostramos solo los primeros 100 caracteres
        print(f"   * Tópico dominante: {topico_dom}")
        print(f"   * Distribución completa de tópicos (id: probabilidad):")
        print(f"     {[(t, round(p, 4)) for t, p in distribucion]}\n")


if __name__ == "__main__":
    main()


ValueError: Usecols do not match columns, columns expected but not found: ['category', 'summit']